# Machine Learning: CNN Architectures

## Example of model combination 

1. ### Get the ResNet50 without the classification part (include_top = false)
2. ### Create a new model that combine the ResNet50 with a simple model

##### Imports

In [ ]:
import os
import datetime
import tensorflow as tf

In [ ]:
import numpy as np
import pylab as pl

##### Downloading the ResNet50

In [ ]:
base_model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3))

##### Creating some layers

In [ ]:
l1 = tf.keras.layers.Conv2DTranspose(64, (3,3), activation="relu", padding="same", strides=2)
l2 = tf.keras.layers.Conv2DTranspose(32, (3,3), activation="relu", padding="same", strides=2)
l3 = tf.keras.layers.Conv2DTranspose(16, (3,3), activation="relu", padding="same", strides=2)
l4 = tf.keras.layers.Conv2DTranspose(8, (3,3), activation="relu", padding="same", strides=2)
l5 = tf.keras.layers.Conv2DTranspose(4, (3,3), activation="relu", padding="same", strides=2)
out = tf.keras.layers.Conv2D(3, (3,3), activation="sigmoid", padding="same")

##### Adding our layers on top of ResNet50

In [ ]:
model = tf.keras.Sequential([
    base_model,
    l1,
    l2,
    l3,
    l4,
    l5,
    out
])

##### Summary

In [ ]:
model.summary()

##### Example

In [ ]:
t = np.random.randn(1,224, 224, 3) # test input, 4 dimensions (bach_size = 1, width, height, channels = 3)
o = model(t) # feed-forward with our custom model
print("o.shape: ", o.shape) 

##### Downloading the dataset (already splitted into train and test)

In [ ]:
 (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar100.load_data()

In [ ]:
# image preprocessing
NB_IMAGES_TO_USE = 1000

train_images = train_images[:NB_IMAGES_TO_USE] / 255.0
test_images = test_images[:NB_IMAGES_TO_USE]  / 255.0

##### Generator

In [ ]:
# Goes through the list of items in "images" 
# and return one value each time the function is called
def generator(images):
    while True:
        for im in images:
            im = tf.image.resize(im, (224, 224))
            noised = im + np.random.normal(0., 0.1, im.shape)
            noised = noised[np.newaxis]
            yield noised, im[np.newaxis]

In [ ]:
train_gen = generator(train_images)

In [ ]:
# Adding an extra dimension 
print("Initial training set shape: ", train_images.shape)

##### Training 

In [ ]:
tf.keras.models.load_model()

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:
# define a folder to store the training data for monitoring
logdir = os.path.join("combine_nets_logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# give the previous folder to Tensorboard 
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

train_gen = generator(train_images)
model.fit(train_gen, epochs=10, steps_per_epoch=20, callbacks=[tensorboard_callback])

##### Testing

In [ ]:
test_gen = generator(test_images)
(test_im, test_gt) = next(test_gen)
outputs = model.predict(test_im)
pl.subplot(121)
pl.imshow(np.squeeze(test_im))
pl.subplot(122)
pl.imshow(np.squeeze(outputs))